# Convert Calcium Imaging data from .mat to NWB file
More details on [NWB Calcium imaging data](https://pynwb.readthedocs.io/en/stable/tutorials/domain/ophys.html#calcium-imaging-data).

**0.** We start importing the relevant modules to read from .mat file and to manipulate NWB file groups and datasets

In [ ]:
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile, NWBHDF5IO, ProcessingModule
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence, DfOverF, MotionCorrection
from pynwb.device import Device
from pynwb.base import TimeSeries

import scipy.io
import numpy as np
import h5py
import os

**1.** Load the .mat files containing calcium imaging data

In [ ]:
path_to_files = ''

# Open info file
fname0 = 'fly2_run1_info.mat'
fpath0 = os.path.join(path_to_files, fname0)
f_info = scipy.io.loadmat(fpath0, struct_as_record=False, squeeze_me=True)
info = f_info['info']

# Open .mat file containing Calcium Imaging data
fname1 = '2019_04_18_Nsyb_NLS6s_Su_walk_G_fly2_run1_8401reg.mat'
fpath1 = os.path.join(path_to_files, fname1)
file = h5py.File(fpath1, 'r')
options = file['options']
landmarkThreshold = file['landmarkThreshold']
templates = file['templates']
R = file['R']
Y = file['Y']

**2.** Create a new [NWB file instance](https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile), fill it with all the relevant information

In [ ]:
#Create new NWB file
nwb = NWBFile(session_description='my CaIm recording', 
              identifier='EXAMPLE_ID', 
              session_start_time=datetime.now(tzlocal()),
              experimenter='Evan Schaffer',
              lab='Axel lab',
              institution='Columbia University',
              experiment_description='EXPERIMENT_DESCRIPTION',
              session_id='IDX')
print(nwb)

**3.** Create [Device](https://pynwb.readthedocs.io/en/stable/pynwb.device.html#pynwb.device.Device) and [OpticalChannel](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.OpticalChannel) containers to be used by a specific [ImagingPlane](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.ImagingPlane).

In [ ]:
#Create and add device
device = Device('Device')
nwb.add_device(device)

# Create an Imaging Plane for Yellow
optical_channel_Y = OpticalChannel(name='OpticalChannel_Y',
                                   description='2P Optical Channel',
                                   emission_lambda=510.)
imaging_plane_Y = nwb.create_imaging_plane(name='ImagingPlane_Y',
                                           optical_channel=optical_channel_Y,
                                           description='Imaging plane',
                                           device=device,
                                           excitation_lambda=488., 
                                           imaging_rate=info.daq.scanRate,
                                           indicator='NLS-GCaMP6s',
                                           location='whole central brain')

# Create an Imaging Plane for Red
optical_channel_R = OpticalChannel(name='OpticalChannel_R',
                                   description='2P Optical Channel',
                                   emission_lambda=633.)
imaging_plane_R = nwb.create_imaging_plane(name='ImagingPlane_R',
                                           optical_channel=optical_channel_R,
                                           description='Imaging plane',
                                           device=device,
                                           excitation_lambda=488., 
                                           imaging_rate=info.daq.scanRate,
                                           indicator='redStinger',
                                           location='whole central brain')

print(nwb)

**4.** Create a [TwoPhotonSeries](https://pynwb.readthedocs.io/en/stable/pynwb.ophys.html#pynwb.ophys.TwoPhotonSeries) container to store the raw data. Raw data usually goes on the `acquisition` group of NWB files.

In [ ]:
#Change dimensions from (X,Y,Z,T) in mat file to (T,X,Y,Z) nwb standard
Y = np.moveaxis(Y, -1, 0)
R = np.moveaxis(R, -1, 0)
print('Y dims: ', Y.shape)
print('R dims: ', R.shape)

#Stores raw data in acquisition group
raw_image_series_Y = TwoPhotonSeries(name='TwoPhotonSeries_Y', 
                                     imaging_plane=imaging_plane_Y,
                                     rate=info.daq.scanRate,
                                     dimension=Y.shape,
                                     data=Y) 

raw_image_series_R = TwoPhotonSeries(name='TwoPhotonSeries_R', 
                                     imaging_plane=imaging_plane_R,
                                     rate=info.daq.scanRate,
                                     dimension=R.shape,
                                     data=R) 

nwb.add_acquisition(raw_image_series_Y)
nwb.add_acquisition(raw_image_series_R)

print(nwb.acquisition)

**5.** The NWB structure is is place, but we still need to save it to file:

In [ ]:
#Saves to NWB file
fname_nwb = 'file_1.nwb'
fpath_nwb = os.path.join(path_to_files, fname_nwb)
with NWBHDF5IO(fpath_nwb, mode='w') as io:
    io.write(nwb)
print('File saved with size: ', os.stat(fpath_nwb).st_size/1e6, ' mb')

**6.** Finally, let's load it and check the file contents:

In [ ]:
#Loads NWB file
with NWBHDF5IO(fpath_nwb, mode='r') as io:
    nwb = io.read()
    print(nwb)